In [1]:
import pandas as pd
import requests
import base64
import re
from datetime import datetime
import os
import time
import json
import random 

# WordPress 사이트 설정
WP_URL = "https://prayer-church.co.kr"  # WordPress 사이트 URL
WP_API = f"{WP_URL}/wp-json/wp/v2"
RANKMATH_API = f"{WP_URL}/wp-json/rankmath/v1"
USERNAME = "prayerchurch"  # WordPress 사용자명
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"  # 응용프로그램 비밀번호

# 오늘 날짜 문자열 생성 (YYYYMMDD 형식)
today_str = datetime.now().strftime('%Y%m%d')
# 파일 경로 설정
EXCEL_PATH = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list_{today_str}.xlsx"
REPORT_PATH = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/upload_report_{today_str}.xlsx"
MEDIA_REPORT_PATH = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{today_str}/report/media_upload_report_{today_str}.xlsx"

# 미디어 업로드 리포트 로드
def load_media_report():
    try:
        media_df = pd.read_excel(MEDIA_REPORT_PATH)
        print(f"미디어 업로드 리포트 로드 성공: {len(media_df)}개 항목")
        return media_df
    except Exception as e:
        print(f"미디어 업로드 리포트 로드 실패: {str(e)}")
        return None

    
# 카테고리에서 관련 글 가져오기 함수 추가
def get_related_posts_in_category(category_id, current_post_id=None, limit=1):
    headers = get_wp_headers()
    
    # 현재 카테고리의 최신 게시물 6개 가져오기 (관련 글 선택용)
    response = requests.get(
        f"{WP_API}/posts", 
        headers=headers,
        params={
            'categories': category_id,
            'per_page': 6,  # 더 많은 글을 가져와서 현재 글 제외하고 선택
            'orderby': 'date',
            'order': 'desc',
            '_embed': 1  # 관련 데이터도 함께 가져오기
        }
    )
    
    if response.status_code != 200:
        print(f"관련 글 가져오기 실패: {response.status_code}")
        return []
    
    posts = response.json()
    
    # 현재 글 제외하고 관련 글 선택
    related_posts = []
    for post in posts:
        # 현재 글이면 건너뜀
        if current_post_id and post['id'] == current_post_id:
            continue
        related_posts.append(post)
        if len(related_posts) >= limit:
            break
    
    return related_posts

# 본문 내용에서 태그 추출 및 처리 함수 
def process_body_content(body_text, title, category_id=None, post_id=None):
    # 본문 내용 복사
    processed_text = body_text
  
    
    # 이미지 업로드 추가 파일명 생성 (타이틀에서 공백을 - 로 변경)
    image_filename = f"banner_{title.replace(' ', '-')}.webp"

    # 오늘 날짜 문자열 생성 (YYYYMMDD 형식)
    today_str = datetime.now().strftime('%Y%m%d')

    # 이미지 경로 생성
    image_path = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{today_str}/images/webp/{image_filename}"

    # 최상단 이미지 HTML 생성
    banner_html = ""

    # 이미지 파일이 있는지 확인
    if os.path.exists(image_path):
        # 이미지 업로드하기
        headers = get_wp_headers()

        # 이미지 데이터 읽기
        with open(image_path, 'rb') as img_file:
            image_data = img_file.read()

        # multipart/form-data 형식으로 이미지 업로드
        files = {
            'file': (image_filename, image_data, 'image/webp')
        }

        # 인증 헤더 준비 (파일 업로드는 multipart/form-data 형식이므로 다른 방식의 인증 사용)
        credentials = f"{USERNAME}:{APP_PASSWORD}"
        token = base64.b64encode(credentials.encode())
        auth_header = {'Authorization': f'Basic {token.decode("utf-8")}'}

        # 이미지 업로드
        upload_response = requests.post(
            f"{WP_API}/media",
            headers=auth_header,
            files=files
        )

        if upload_response.status_code == 201:
            # 업로드된 이미지 정보 가져오기
            image_data = upload_response.json()
            image_id = image_data['id']
            image_url = image_data['source_url']

            # 배너 이미지 HTML 생성
            banner_html = f'''<!-- wp:image {{"align":"wide","id":{image_id},"sizeSlug":"full","linkDestination":"none"}} -->
    <figure class="wp-block-image alignwide size-full"><img src="{image_url}" alt="{title}" class="wp-image-{image_id}"/></figure>
    <!-- /wp:image -->

    '''
            print(f"배너 이미지 업로드 성공: {image_filename}")
        else:
            print(f"배너 이미지 업로드 실패: {upload_response.status_code} - {upload_response.text}")
    else:
        print(f"배너 이미지 파일을 찾을 수 없습니다: {image_path}")
    
    # 헤딩 태그 추출 및 처리 (h2, h3, h4, h5, h6)
    heading_tags = re.findall(r'(h[2-6]:)([^\n]+)', processed_text)
    for tag, content in heading_tags:
        tag_level = tag[1]  # h2의 경우 2를 가져옴
        processed_text = processed_text.replace(f"{tag}{content}", f"<h{tag_level}>{content.strip()}</h{tag_level}>")
    
        # 줄 간격을 넓히기 위한 CSS 스타일 추가 (본문 시작 부분에 추가)
    # WordPress의 콘텐츠 영역에 스타일 적용
    line_spacing_style = """<!-- wp:html -->
<style>
  .entry-content p, 
  .entry-content li,
  .wp-block-paragraph {
    line-height: 2 !important; /* 줄 간격을 2배로 설정 */
    margin-bottom: 1.5em !important; /* 문단 간격도 키움 */
  }
  
  /* 모바일 디바이스에서도 적절한 줄 간격 유지 */
  @media (max-width: 768px) {
    .entry-content p, 
    .entry-content li,
    .wp-block-paragraph {
      line-height: 1.8 !important;
      margin-bottom: 1.3em !important;
    }
  }
</style>
<!-- /wp:html -->

"""
    
    # 본문 텍스트를 HTML 단락 태그로 감싸기
    # 우선 대규모 단락으로 나누기 (빈 줄을 기준으로)
    # 본문 텍스트를 HTML 단락 태그로 감싸기
    paragraphs = processed_text.split('\n\n')  # 빈 줄을 기준으로 큰 단락 분리
    processed_html = ""

    for paragraph in paragraphs:
        paragraph = paragraph.strip()
        if not paragraph:
            continue

        # 이미 HTML 태그로 감싸진 경우는 그대로 유지
        if paragraph.startswith('<') and paragraph.endswith('>'):
            processed_html += paragraph + "\n\n"
        # 헤딩 태그가 있는 경우도 그대로 유지
        elif re.match(r'<h[2-6]>', paragraph):
            processed_html += paragraph + "\n\n"
        else:
            # 단일 줄바꿈 처리 - 각 줄을 <p> 태그로 래핑
            lines = paragraph.split('\n')
            for line in lines:
                line = line.strip()
                if line:
                    processed_html += f"<!-- wp:paragraph -->\n<p>{line}</p>\n<!-- /wp:paragraph -->\n\n"

    # 스타일 태그를 본문 앞에 추가
    # 배너 이미지와 스타일 태그를 본문 앞에 추가
    processed_text = banner_html + line_spacing_style + processed_html    
    # 미디어 리포트 로드
    media_df = load_media_report()
    
    # 나머지 코드는 그대로 유지...
    
    # 미디어 리포트가 없는 경우 간단한 HTML 블록 추가
    if media_df is None:
        # 랜덤으로 텍스트 선택
        suffix_options = [
            "다운로드 하기",
            "다운로드",
            "다운로드 받기"
        ]
        selected_suffix = random.choice(suffix_options)
        
        custom_html = f"""<h4 style="border-left: 5px solid #B00808; background-color: #f8f4f4; padding: 12px 15px; font-weight: bold; margin: 20px 0; position: relative; color: #333;">{title} {selected_suffix}</h4>"""
        processed_text += custom_html
        return processed_text
    
    # # 와 ** 문자 제거
    processed_text = processed_text.replace('#', '').replace('**', '')
    
    # 미디어 리포트 로드
    media_df = load_media_report()
    
    # 미디어 리포트가 없는 경우 간단한 HTML 블록 추가
    if media_df is None:
        # 랜덤으로 텍스트 선택
        suffix_options = [
            "다운로드 하기",
            "다운로드",
            "다운로드 받기"
        ]
        selected_suffix = random.choice(suffix_options)
        
        custom_html = f"""<h4 style="border-left: 5px solid #B00808; background-color: #f8f4f4; padding: 12px 15px; font-weight: bold; margin: 20px 0; position: relative; color: #333;">{title} {selected_suffix}</h4>"""
        processed_text += custom_html
        return processed_text
    
    # 제목 키워드로 파일 찾기
    matching_rows = []
    title_keywords = title.split()
    
    for index, row in media_df.iterrows():
        if '제목' not in row or pd.isna(row['제목']):
            continue
            
        file_title = str(row['제목'])
        match_score = 0
        
        # 제목 키워드 매칭 점수 계산
        for keyword in title_keywords:
            if len(keyword) > 1 and keyword in file_title:  # 2글자 이상 키워드만 고려
                match_score += 1
        
        if match_score > 0:
            matching_rows.append((row, match_score))
    
    # 매칭 점수가 높은 순으로 정렬
    matching_rows.sort(key=lambda x: x[1], reverse=True)
    
    # 매칭된 파일이 없으면 간단한 HTML 블록 추가
    if not matching_rows:
        suffix_options = ["다운로드 하기", "다운로드", "다운로드 받기"]
        selected_suffix = random.choice(suffix_options)
        
        custom_html = f"""<h4 style="border-left: 5px solid #B00808; background-color: #f8f4f4; padding: 12px 15px; font-weight: bold; margin: 20px 0; position: relative; color: #333;">{title} {selected_suffix}</h4>"""
        processed_text += custom_html
        return processed_text
    
    # 가장 매칭 점수가 높은 파일 선택
    best_match = matching_rows[0][0]
    
    # URL 가져오기
    word_url = best_match['URL'] if 'URL' in best_match else ''
    
    # PDF URL 찾기 (동일한 파일명의 PDF 버전)
    pdf_url = ''
    best_match_title = best_match['제목'] if '제목' in best_match else ''
    
    for index, row in media_df.iterrows():
        if '제목' not in row or pd.isna(row['제목']):
            continue
            
        if row['제목'] == best_match_title and 'URL' in row and '.pdf' in row['URL']:
            pdf_url = row['URL']
            break
    
    # PDF URL이 없으면 Word URL에서 확장자만 변경
    if not pdf_url and word_url and '.docx' in word_url:
        pdf_url = word_url.replace('.docx', '.pdf')
    
    # 랜덤으로 텍스트 선택
    suffix_options = [
        "다운로드 하기",
        "다운로드",
        "다운로드 받기"
    ]
    selected_suffix = random.choice(suffix_options)
    
    # 다운로드 제목 생성
    download_title = f"{title} {selected_suffix}"
    
    # 헤더 HTML 생성 (HTML 블록으로 추가)
    header_html = f'''<!-- wp:html -->
<h4 style="border-left: 5px solid #B00808; background-color: #f8f4f4; padding: 12px 15px; font-weight: bold; margin: 20px 0; position: relative; color: #333;">{download_title}</h4>
<!-- /wp:html -->'''
    
    # 인라인 HTML 코드 생성 - 광고 삽입 방지 코드 추가
    download_html = f'''<!-- wp:html -->
<!-- no-adsense-start -->
<div style="max-width: 650px; margin: 30px auto; font-family: 'Segoe UI', Arial, sans-serif;" aria-label="기도문 다운로드 섹션" role="region" class="no-ads-zone">
  <div style="text-align: center; margin-bottom: 25px;">
    <h2 style="color: #333; font-size: 22px; margin-bottom: 10px;">{download_title}</h2>
    <p style="color: #666; font-size: 16px; margin: 0;">마음의 깊이를 더하는 기도문을 다운로드하세요</p>
  </div>

  <div style="text-align: center; margin-bottom: 20px;">
    <a href="{word_url}" 
       style="display: inline-block; text-decoration: none; font-weight: bold; color: #333; background: linear-gradient(to bottom, #f8f8f8, #f1f1f1); padding: 14px 30px; border-radius: 8px; border: 1px solid #ddd; box-shadow: 0 3px 8px rgba(0,0,0,0.08); transition: all 0.3s ease;"
       title="{title} 다운로드" 
       aria-label="{title} 워드 파일 다운로드"
       download
       rel="nofollow">
      <span style="color: #B00808; font-size: 18px; margin-right: 8px;" aria-hidden="true">▼</span>
      <span style="font-size: 16px;">{title} 다운로드</span>
      <span style="color: #B00808; font-size: 18px; margin-left: 8px;" aria-hidden="true">▼</span>
    </a>
  </div>

  <div style="text-align: center; margin-bottom: 20px;">
    <p style="color: #777; font-size: 14px;" id="file-options-desc">원하시는 파일 형식을 선택하세요</p>
  </div>

  <div style="display: flex; flex-direction: column; gap: 16px;" aria-describedby="file-options-desc" role="group" aria-label="파일 형식 옵션">
    <!-- 워드 파일 다운로드 -->
    <a href="{word_url}" 
       style="display: flex; align-items: center; justify-content: space-between; text-decoration: none; border: 1px solid #e8e8e8; border-radius: 12px; padding: 16px 24px; background: linear-gradient(to right, #fcfcfc, #f7f7f7); box-shadow: 0 3px 10px rgba(0,0,0,0.06); transition: all 0.3s ease;"
       title="{title} 워드 파일 다운로드" 
       aria-label="워드 문서 형식으로 기도문 다운로드"
       download
       data-filetype="word"
       data-filesize="0.07MB"
       rel="nofollow">
      <div style="display: flex; align-items: center;">
        <div style="width: 42px; height: 42px; background: linear-gradient(135deg, #2B5797, #1E3C64); border-radius: 8px; margin-right: 18px; display: flex; align-items: center; justify-content: center; box-shadow: 0 2px 5px rgba(0,0,0,0.15);" aria-hidden="true">
          <svg width="22" height="22" viewBox="0 0 24 24" fill="#fff" aria-hidden="true">
            <path d="M14,2H6A2,2 0 0,0 4,4V20A2,2 0 0,0 6,22H18A2,2 0 0,0 20,20V8L14,2M18,20H6V4H13V9H18V20M9.5,11A0.5,0.5 0 0,1 10,11.5V13H14V11.5A0.5,0.5 0 0,1 14.5,11A0.5,0.5 0 0,1 15,11.5V13A1,1 0 0,1 14,14H10A1,1 0 0,1 9,13V11.5A0.5,0.5 0 0,1 9.5,11M10,15H14A1,1 0 0,1 15,16V18H14V18H10V18H9V16A1,1 0 0,1 10,15Z" />
          </svg>
        </div>
        <div>
          <div style="font-weight: 600; color: #333; font-size: 16px; margin-bottom: 5px; letter-spacing: -0.3px;">{title}</div>
          <div style="display: flex; align-items: center; gap: 10px;">
            <span style="background-color: #f0f0f0; padding: 3px 8px; border-radius: 20px; font-size: 12px; color: #555;">Word 문서</span>
            <span style="background-color: #f0f0f0; padding: 3px 8px; border-radius: 20px; font-size: 12px; color: #555;">0.07MB</span>
          </div>
        </div>
      </div>
      <div style="display: flex; align-items: center; justify-content: center; width: 44px; height: 44px; background-color: #2B5797; border-radius: 50%; transition: all 0.3s ease; box-shadow: 0 2px 6px rgba(43, 87, 151, 0.2);" aria-hidden="true">
        <svg width="20" height="20" viewBox="0 0 24 24" fill="#fff" aria-label="다운로드 아이콘">
          <path d="M19,12L12,19L5,12H8V6H16V12H19Z" />
        </svg>
      </div>
    </a>

    <!-- PDF 파일 다운로드 -->
    <a href="{pdf_url if pdf_url else word_url.replace('.docx', '.pdf') if '.docx' in word_url else word_url}" 
       style="display: flex; align-items: center; justify-content: space-between; text-decoration: none; border: 1px solid #e8e8e8; border-radius: 12px; padding: 16px 24px; background: linear-gradient(to right, #fcfcfc, #f7f7f7); box-shadow: 0 3px 10px rgba(0,0,0,0.06); transition: all 0.3s ease;"
       title="{title} PDF 파일 다운로드" 
       aria-label="PDF 문서 형식으로 기도문 다운로드"
       download
       data-filetype="pdf"
       data-filesize="0.07MB"
       rel="nofollow">
      <div style="display: flex; align-items: center;">
        <div style="width: 42px; height: 42px; background: linear-gradient(135deg, #B00808, #910707); border-radius: 8px; margin-right: 18px; display: flex; align-items: center; justify-content: center; box-shadow: 0 2px 5px rgba(0,0,0,0.15);" aria-hidden="true">
          <svg width="22" height="22" viewBox="0 0 24 24" fill="#fff" aria-hidden="true">
            <path d="M19,3A2,2 0 0,1 21,5V19A2,2 0 0,1 19,21H5C3.89,21 3,20.1 3,19V5C3,3.89 3.89,3 5,3H19M18.5,18.5V5.5H5.5V18.5H18.5M9.5,11A0.5,0.5 0 0,0 9,11.5V13H15V11.5A0.5,0.5 0 0,0 14.5,11A0.5,0.5 0 0,0 14,11.5V12H10V11.5A0.5,0.5 0 0,0 9.5,11M11,14H13A1,1 0 0,1 14,15V16H13V15H11V16H10V15A1,1 0 0,1 11,14Z" />
          </svg>
        </div>
        <div>
          <div style="font-weight: 600; color: #333; font-size: 16px; margin-bottom: 5px; letter-spacing: -0.3px;">{title}</div>
          <div style="display: flex; align-items: center; gap: 10px;">
            <span style="background-color: #f0f0f0; padding: 3px 8px; border-radius: 20px; font-size: 12px; color: #555;">PDF 문서</span>
            <span style="background-color: #f0f0f0; padding: 3px 8px; border-radius: 20px; font-size: 12px; color: #555;">0.07MB</span>
          </div>
        </div>
      </div>
      <div style="display: flex; align-items: center; justify-content: center; width: 44px; height: 44px; background-color: #B00808; border-radius: 50%; transition: all 0.3s ease; box-shadow: 0 2px 6px rgba(176, 8, 8, 0.2);" aria-hidden="true">
        <svg width="20" height="20" viewBox="0 0 24 24" fill="#fff" aria-label="다운로드 아이콘">
          <path d="M19,12L12,19L5,12H8V6H16V12H19Z" />
        </svg>
      </div>
    </a>
  </div>

  <div style="text-align: center; margin-top: 25px;">
    <p style="color: #888; font-size: 13px; margin: 0;">위 파일은 자유롭게 다운로드하여 사용하실 수 있습니다</p>
    <p style="color: #999; font-size: 12px; margin-top: 10px; max-width: 80%; margin-left: auto; margin-right: auto;">
      <small>대표 기도문 나눔터의 {title} 자료입니다. 예배, 소그룹 모임, 개인 기도에 활용하실 수 있습니다.</small>
    </p>
  </div>
  
  <span style="position: absolute; width: 1px; height: 1px; padding: 0; margin: -1px; overflow: hidden; clip: rect(0, 0, 0, 0); white-space: nowrap; border: 0;" aria-hidden="true">
    예배 기도문, 주일 대표 기도, 대표 기도문, 기도, 기도문 예시, 대표기도 예문
  </span>
</div>

<style>
  .no-ads-zone a:hover {{
    transform: translateY(-2px);
    box-shadow: 0 5px 15px rgba(0,0,0,0.1);
  }}
  
  .no-ads-zone a:focus {{
    outline: 2px solid #B00808;
    outline-offset: 2px;
  }}
  
  @media (hover: hover) {{
    .no-ads-zone a:hover {{
      transform: translateY(-2px);
      box-shadow: 0 5px 15px rgba(0,0,0,0.1);
    }}
  }}
  
  @media (max-width: 480px) {{
    .no-ads-zone div[style*="max-width: 650px"] {{
      padding: 0 15px;
    }}
  }}
</style>

<script type="application/ld+json">
{{
  "@context": "https://schema.org",
  "@type": "DigitalDocument",
  "name": "{title}",
  "description": "대표 기도문 나눔터의 {title} 입니다.",
  "datePublished": "{datetime.now().strftime('%Y-%m')}",
  "encodingFormat": ["application/pdf", "application/vnd.openxmlformats-officedocument.wordprocessingml.document"],
  "about": {{
    "@type": "Thing",
    "name": "대표 기도문 나눔터"
  }},
  "offers": {{
    "@type": "Offer",
    "price": "0",
    "priceCurrency": "KRW",
    "availability": "https://schema.org/InStock"
  }}
}}
</script>
<!-- no-adsense-end -->
<!-- /wp:html -->'''

    # 카테고리에서 관련 글 가져오기
    related_posts_html = ""
    if category_id:
        related_posts = get_related_posts_in_category(category_id, post_id, limit=1)
        
        for related_post in related_posts:
            # 관련 글 정보 추출
            related_title = related_post.get('title', {}).get('rendered', '관련 기도문')
            related_excerpt = related_post.get('excerpt', {}).get('rendered', '')
            # HTML 태그 제거
            related_excerpt = re.sub(r'<[^>]+>', '', related_excerpt)
            # 텍스트 길이 제한
            if len(related_excerpt) > 120:
                related_excerpt = related_excerpt[:120] + '...'
                
            related_url = related_post.get('link', '')
            related_date = datetime.fromisoformat(related_post.get('date', '').replace('Z', '+00:00')).strftime('%Y년')
            
            # 관련 글 제목 추출 (주로 첫 부분에 있는 경우가 많음)
            related_post_main_title = related_title
            match = re.search(r'(.+?)(-|–|—)', related_title)
            if match:
                related_post_main_title = match.group(1).strip()
            
            # 링크 카드 HTML 생성
            related_posts_html += f'''<!-- wp:html -->
<div itemscope itemtype="https://schema.org/Article" class="link-card">
  <a href="{related_url}" style="display: flex; max-width: 100%; width: 100%; margin: 0 auto; text-decoration: none; border-radius: 12px; overflow: hidden; box-shadow: 0 2px 10px rgba(0,0,0,0.1); transition: all 0.3s ease; background-color: #fff;" rel="noopener" target="_blank">
    
    <!-- 왼쪽 섬네일 영역 - 항상 왼쪽에 배치 -->
    <div style="width: 130px; min-width: 130px; background-color: #F8F8F8; position: relative; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 15px 10px; text-align: center; border-right: 3px solid #B00808;">
      <div style="font-weight: 800; font-size: 18px; color: #000; line-height: 1.2; margin-bottom: 8px;">
        <div style="color: #000;">{related_post_main_title}</div>
        <div style="margin-top: 4px; color: #B00808;">대표기도문</div>
      </div>
      <div style="font-weight: 600; font-size: 16px; color: #333; margin-top: 10px;">{related_date}</div>
    </div>
    
    <!-- 오른쪽 콘텐츠 영역 -->
    <div style="flex: 1; padding: 15px; background-color: #fff; display: flex; flex-direction: column; justify-content: center; min-width: 0;">
      <!-- 제목 -->
      <h3 itemprop="headline" style="margin: 0 0 10px; font-size: 16px; font-weight: 700; color: #000; line-height: 1.3; letter-spacing: -0.5px; white-space: normal; word-break: break-word;">
        {related_title}
      </h3>
      
      <!-- 설명 -->
      <p itemprop="description" style="margin: 0 0 10px; font-size: 13px; color: #333; line-height: 1.4; display: -webkit-box; -webkit-line-clamp: 2; -webkit-box-orient: vertical; overflow: hidden; text-overflow: ellipsis; letter-spacing: -0.3px; word-break: break-word;">
        {related_excerpt}
      </p>
      
      <!-- 하단 정보 -->
      <div style="display: flex; justify-content: space-between; align-items: center; margin-top: 8px;">
        <div itemprop="publisher" itemscope itemtype="https://schema.org/Organization" style="font-size: 12px; color: #333; display: flex; align-items: center; overflow: hidden; text-overflow: ellipsis; white-space: nowrap; max-width: 65%;">
          <svg xmlns="http://www.w3.org/2000/svg" width="12" height="12" viewBox="0 0 24 24" fill="none" stroke="#333" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" style="margin-right: 4px; min-width: 12px; flex-shrink: 0;">
            <path d="M10 13a5 5 0 0 0 7.54.54l3-3a5 5 0 0 0-7.07-7.07l-1.72 1.71"></path>
            <path d="M14 11a5 5 0 0 0-7.54-.54l-3 3a5 5 0 0 0 7.07 7.07l1.71-1.71"></path>
          </svg>
          <span itemprop="name" style="overflow: hidden; text-overflow: ellipsis;">prayer-church.co.kr</span>
        </div>
        
        <!-- 바로가기 버튼 -->
        <div class="goto-button" style="background-color: #B00808; padding: 6px 12px; border-radius: 50px; font-size: 12px; color: #fff; font-weight: 500; display: flex; align-items: center; transition: all 0.2s ease; white-space: nowrap; flex-shrink: 0;">
          <span>바로가기</span>
          <svg xmlns="http://www.w3.org/2000/svg" width="12" height="12" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" style="margin-left: 4px; min-width: 12px;">
            <path d="M5 12h14"></path>
            <path d="M12 5l7 7-7 7"></path>
          </svg>
        </div>
      </div>
    </div>
  </a>
</div>

<script type="application/ld+json">
{{
  "@context": "https://schema.org",
  "@type": "Article",
  "headline": "{related_title}",
  "description": "{related_excerpt}",
  "publisher": {{
    "@type": "Organization",
    "name": "대표 기도문 나눔터",
    "url": "https://prayer-church.co.kr"
  }},
  "mainEntityOfPage": {{
    "@type": "WebPage",
    "@id": "{related_url}"
  }}
}}
</script>
<style>
.link-card a {{
  transition: all 0.3s ease;
}}
.link-card a:hover {{
  transform: translateY(-2px);
  box-shadow: 0 5px 15px rgba(176, 8, 8, 0.2);
}}
.link-card a:hover .goto-button {{
  background-color: #FF0000 !important;
  transform: translateX(3px);
}}
.link-card a:hover > div:first-child {{
  background-color: #F5F5F5 !important;
}}
.link-card a:hover > div:first-child div div:nth-child(2) {{
  color: #FF0000 !important;
}}
.link-card a:focus {{
  outline: 3px solid #B00808;
  outline-offset: 2px;
}}
@media (min-width: 641px) {{
  .link-card a {{
    max-width: 800px !important;
  }}
  .link-card a > div:first-child {{
    width: 280px !important;
    min-width: 280px !important;
    padding: 30px 20px !important;
  }}
  .link-card a > div:first-child div:nth-child(1) {{
    font-size: 28px !important;
    margin-bottom: 15px !important;
  }}
  .link-card a > div:first-child div:nth-child(2) {{
    font-size: 22px !important;
    margin-top: 15px !important;
  }}
  .link-card a > div:last-child {{
    padding: 30px !important;
  }}
  .link-card a > div:last-child h3 {{
    font-size: 22px !important;
    margin-bottom: 15px !important;
  }}
  .link-card a > div:last-child p {{
    font-size: 16px !important;
    line-height: 1.6 !important;
    -webkit-line-clamp: 3 !important;
    margin-bottom: 20px !important;
  }}
  .link-card a > div:last-child div {{
    margin-top: 15px !important;
  }}
  .link-card a > div:last-child div:first-child {{
    font-size: 15px !important;
    max-width: 80% !important;
  }}
  .link-card a > div:last-child div:first-child svg {{
    width: 16px !important;
    height: 16px !important;
    margin-right: 8px !important;
  }}
  .link-card a > div:last-child div:last-child {{
    font-size: 14px !important;
    padding: 8px 16px !important;
  }}
  .link-card a > div:last-child div:last-child svg {{
    width: 14px !important;
    height: 14px !important;
  }}
}}
.link-card a {{
  font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, "Noto Sans", sans-serif, "Apple Color Emoji", "Segoe UI Emoji", "Segoe UI Symbol", "Noto Color Emoji";
}}
@media (prefers-color-scheme: dark) {{
  .link-card a > div:last-child {{
    background-color: #222;
  }}
  .link-card h3 {{
    color: #fff;
  }}
  .link-card p {{
    color: #ddd;
  }}
  .link-card a > div:first-child {{
    background-color: #333;
  }}
  .link-card a > div:first-child div:nth-child(1) div:nth-child(1) {{
    color: #fff;
  }}
  .link-card a > div:first-child div:nth-child(2) {{
    color: #ddd;
  }}
  .link-card a > div:last-child div:first-child {{
    color: #ddd;
  }}
  .link-card a > div:last-child div:first-child svg {{
    stroke: #ddd;
  }}
  .link-card a:hover > div:first-child {{
    background-color: #444 !important;
  }}
}}
</style>
<!-- /wp:html -->'''


    
    # 본문에 HTML 블록 추가 (각 요소를 독립적인 블록으로 추가)
    processed_text += header_html + download_html + related_posts_html
    
    return processed_text


# WordPress API 인증 설정
def get_wp_headers():
    credentials = f"{USERNAME}:{APP_PASSWORD}"
    token = base64.b64encode(credentials.encode())
    return {
        'Authorization': f'Basic {token.decode("utf-8")}',
        'Content-Type': 'application/json'
    }

# 카테고리 ID 가져오기
def get_category_id(category_name):
    response = requests.get(f"{WP_API}/categories", params={'search': category_name})
    categories = response.json()
    
    if categories:
        for category in categories:
            if category['name'].lower() == category_name.lower():
                return category['id']
    
    # 카테고리가 없으면 새로 생성
    headers = get_wp_headers()
    response = requests.post(
        f"{WP_API}/categories", 
        headers=headers,
        json={'name': category_name}
    )
    
    if response.status_code == 201:
        return response.json()['id']
    else:
        print(f"카테고리 생성 실패: {category_name}")
        return None

# 태그 ID 가져오기
def get_tag_ids(tag_names):
    if not tag_names:
        return []
    
    # 쉼표로 구분된 태그를 리스트로 변환
    if isinstance(tag_names, str):
        tag_list = [tag.strip() for tag in tag_names.split(',')]
    else:
        tag_list = [tag_names]
    
    tag_ids = []
    headers = get_wp_headers()
    
    for tag_name in tag_list:
        if not tag_name:
            continue
            
        # 태그 검색
        response = requests.get(f"{WP_API}/tags", params={'search': tag_name})
        tags = response.json()
        
        tag_id = None
        if tags:
            for tag in tags:
                if tag['name'].lower() == tag_name.lower():
                    tag_id = tag['id']
                    break
        
        # 태그가 없으면 새로 생성
        if not tag_id:
            response = requests.post(
                f"{WP_API}/tags", 
                headers=headers,
                json={'name': tag_name}
            )
            
            if response.status_code == 201:
                tag_id = response.json()['id']
            else:
                print(f"태그 생성 실패: {tag_name}")
                continue
        
        tag_ids.append(tag_id)
    
    return tag_ids

# 다중 접근법을 사용하여 Rank Math SEO 메타데이터 설정
def set_rankmath_metadata(post_id, meta_description, focus_keyword):
    headers = get_wp_headers()
    success = False
    
    # 접근법 1: 표준 WordPress Meta API 사용
    meta_data = {}
    if meta_description:
        meta_data['_rank_math_description'] = meta_description
    if focus_keyword:
        meta_data['_rank_math_focus_keyword'] = focus_keyword
    
    if meta_data:
        for key, value in meta_data.items():
            # WordPress 메타 API를 사용하여 각 메타 필드 개별 설정
            response = requests.post(
                f"{WP_API}/posts/{post_id}/meta",
                headers=headers,
                json={'key': key, 'value': value}
            )
            if response.status_code not in [200, 201]:
                print(f"메타 설정 실패 (접근법 1): {key} - {response.status_code}")
            else:
                success = True
                print(f"메타 설정 성공 (접근법 1): {key}")
    
    # 접근법 2: Rank Math 전용 API 사용
    rank_meta = {}
    if meta_description:
        rank_meta['rank_math_description'] = meta_description
    if focus_keyword:
        rank_meta['rank_math_focus_keyword'] = focus_keyword
    rank_meta['rank_math_robots'] = 'index,follow'
    
    try:
        response = requests.post(
            f"{RANKMATH_API}/updateMeta",
            headers=headers,
            json={
                'objectType': 'post',
                'objectID': post_id,
                'meta': rank_meta
            }
        )
        
        if response.status_code in [200, 201]:
            success = True
            print(f"Rank Math API 메타 설정 성공 (접근법 2)")
        else:
            print(f"Rank Math API 메타 설정 실패 (접근법 2): {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Rank Math API 호출 오류 (접근법 2): {str(e)}")
    
    # 접근법 3: 직접 WordPress 포스트 업데이트를 통한 메타 설정
    direct_meta = {}
    if meta_description:
        direct_meta['rank_math_description'] = meta_description
    if focus_keyword:
        direct_meta['rank_math_focus_keyword'] = focus_keyword
        # 언더스코어로 시작하는 키도 추가 시도
        direct_meta['_rank_math_focus_keyword'] = focus_keyword
    
    try:
        response = requests.post(
            f"{WP_API}/posts/{post_id}",
            headers=headers,
            json={
                'meta': direct_meta
            }
        )
        
        if response.status_code in [200, 201]:
            success = True
            print(f"직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)")
        else:
            print(f"직접 포스트 업데이트를 통한 메타 설정 실패 (접근법 3): {response.status_code} - {response.text}")
    except Exception as e:
        print(f"포스트 업데이트 오류 (접근법 3): {str(e)}")
    
    # 접근법 4: 포커스 키워드를 JSON 문자열로 설정 시도
    if focus_keyword:
        try:
            # 포커스 키워드를 JSON 형식으로 설정
            json_focus_keyword = json.dumps(focus_keyword)
            
            response = requests.post(
                f"{WP_API}/posts/{post_id}/meta",
                headers=headers,
                json={'key': 'rank_math_focus_keyword', 'value': json_focus_keyword}
            )
            
            if response.status_code in [200, 201]:
                success = True
                print(f"JSON 형식 포커스 키워드 설정 성공 (접근법 4)")
            else:
                print(f"JSON 형식 포커스 키워드 설정 실패 (접근법 4): {response.status_code} - {response.text}")
        except Exception as e:
            print(f"JSON 포커스 키워드 설정 오류 (접근법 4): {str(e)}")
    
    # 디버깅을 위해 현재 포스트의 메타데이터 조회
    try:
        response = requests.get(
            f"{WP_API}/posts/{post_id}?context=edit",
            headers=headers
        )
        
        if response.status_code == 200:
            post_data = response.json()
            if 'meta' in post_data:
                print(f"현재 포스트 메타데이터: {json.dumps(post_data['meta'], indent=2)}")
            else:
                print("포스트에 메타데이터가 없습니다")
    except Exception as e:
        print(f"메타데이터 조회 오류: {str(e)}")
    
    return success

# 게시물 업로드 함수
# 게시물 업로드 함수
def upload_post(row):
    headers = get_wp_headers()
    
    # 카테고리 ID 가져오기
    category_id = None
    if 'category' in row and pd.notna(row['category']):
        category_id = get_category_id(row['category'])
    
    # 태그 ID 가져오기
    tag_ids = []
    if 'tag' in row and pd.notna(row['tag']):
        tag_ids = get_tag_ids(row['tag'])
    
    # 메타 설명과 포커스 키워드
    meta_description = row['description'] if 'description' in row and pd.notna(row['description']) else ''
    focus_keyword = row['focus keyword'] if 'focus keyword' in row and pd.notna(row['focus keyword']) else ''
    
    # 게시물 데이터 준비 (본문은 이 단계에서는 처리하지 않음)
    post_data = {
        'title': row['title'],
        'status': 'publish'
    }
    
    # 카테고리 추가
    if category_id:
        post_data['categories'] = [category_id]
    
    # 태그 추가
    if tag_ids:
        post_data['tags'] = tag_ids

    # 접근법 5: 포스트 생성 시에 메타데이터도 함께 설정 시도
    if meta_description or focus_keyword:
        meta_data = {}
        if meta_description:
            meta_data['rank_math_description'] = meta_description
        if focus_keyword:
            meta_data['rank_math_focus_keyword'] = focus_keyword
            # 언더스코어 버전도 함께 시도
            meta_data['_rank_math_focus_keyword'] = focus_keyword

        post_data['meta'] = meta_data

    # 우선 게시물 초안을 생성하여 ID 획득
    response = requests.post(
        f"{WP_API}/posts", 
        headers=headers,
        json=post_data
    )

    if response.status_code == 201:
        post_data = response.json()
        post_id = post_data['id']
        
        # 이제 본문 내용 처리 (post_id가 있으므로 관련 글 찾기 가능)
        body_content = process_body_content(
            row['body'], 
            row['title'], 
            category_id=category_id, 
            post_id=post_id
        ) if 'body' in row and pd.notna(row['body']) else ''
        
        # 게시물 업데이트
        update_response = requests.post(
            f"{WP_API}/posts/{post_id}", 
            headers=headers,
            json={'content': body_content, 'status': 'publish'}
        )
        
        if update_response.status_code != 200:
            print(f"게시물 내용 업데이트 실패: {update_response.status_code}")

        # 게시물 생성 후 다시 메타데이터 설정 (여러 접근법 시도)
        if meta_description or focus_keyword:
            meta_success = set_rankmath_metadata(post_id, meta_description, focus_keyword)
            if not meta_success:
                print(f"경고: 포스트 ID {post_id}의 Rank Math 메타데이터 설정에 실패했습니다.")

        return {
            'status': 'success',
            'post_id': post_id,
            'post_url': post_data['link'],
            'message': '업로드 성공'
        }
    else:
        error_message = f'업로드 실패: {response.status_code}'
        try:
            error_json = response.json()
            error_message += f" - {json.dumps(error_json)}"
        except:
            error_message += f" - {response.text}"

        return {
            'status': 'error',
            'post_id': None,
            'post_url': None,
            'message': error_message
        }
        

# 메인 함수 - upload_post 함수 외부에 위치해야 함
def main():
    # 미디어 리포트 존재 여부 확인
    try:
        media_df = pd.read_excel(MEDIA_REPORT_PATH)
        print(f"미디어 업로드 리포트 로드 성공: {len(media_df)}개 항목")
    except Exception as e:
        print(f"미디어 업로드 리포트 로드 실패: {str(e)}")
        print("미디어 리포트 없이 진행합니다.")
        media_df = None

    # 엑셀 파일 로드
    try:
        df = pd.read_excel(EXCEL_PATH)
        print(f"총 {len(df)} 개의 게시물을 처리합니다.")
    except Exception as e:
        print(f"엑셀 파일 로드 실패: {str(e)}")
        return

    # 결과를 저장할 리스트
    results = []

    # 각 행 처리
    for index, row in df.iterrows():
        print(f"게시물 {index+1}/{len(df)} 처리 중: {row['title']}")

        # 게시물 업로드
        result = upload_post(row)

        # 원본 데이터와 결과 합치기
        row_data = row.to_dict()
        row_data.update(result)
        results.append(row_data)

        # 워드프레스 서버에 과부하를 주지 않기 위해 잠시 대기
        time.sleep(2)

    # 결과 저장
    result_df = pd.DataFrame(results)
    result_df.to_excel(REPORT_PATH, index=False)
    print(f"업로드 완료. 결과 리포트가 저장되었습니다: {REPORT_PATH}")

if __name__ == "__main__":
    main()

미디어 업로드 리포트 로드 성공: 150개 항목
총 80 개의 게시물을 처리합니다.
게시물 1/80 처리 중: 새벽예배 대표기도문 10월
배너 이미지 업로드 성공: banner_새벽예배-대표기도문-10월.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 2/80 처리 중: 새벽예배 대표기도문 11월
배너 이미지 업로드 성공: banner_새벽예배-대표기도문-11월.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55

현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 15/80 처리 중: 새벽예배 대표기도문 8월 셋째주
태그 생성 실패: 새벽예배 대표기도문
태그 생성 실패: 새벽예배 기도문
배너 이미지 업로드 성공: banner_새벽예배-대표기도문-8월-셋째주.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 16/80 처리 중: 새벽예배 대표기도문 8월 넷째주
태그 생성 실패: 새벽예배 대표기도문
태그 생성 실패: 새벽예배 대표기도
배너 이미지 업로드 성공: banner_새벽예배-대표기도문-8월-넷째주.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드

현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 29/80 처리 중: 새벽예배 대표기도문 10월 둘째주
태그 생성 실패: 새벽예배 대표기도문
배너 이미지 업로드 성공: banner_새벽예배-대표기도문-10월-둘째주.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 30/80 처리 중: 새벽예배 대표기도문 10월 셋째주
태그 생성 실패: 새벽예배 대표기도문
배너 이미지 업로드 성공: banner_새벽예배-대표기도문-10월-셋째주.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest

현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 43/80 처리 중: 6월 1째주 주일 낮예배 대표 기도문
배너 이미지 업로드 성공: banner_6월-1째주-주일-낮예배-대표-기도문.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 44/80 처리 중: 6월 2째주 주일 낮예배 대표 기도문
배너 이미지 업로드 성공: banner_6월-2째주-주일-낮예배-대표-기도문.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \u

배너 이미지 업로드 성공: banner_9월-셋째주-주일-낮예배-대표-기도문.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 58/80 처리 중: 9월 넷째주 주일 낮예배 대표 기도문
배너 이미지 업로드 성공: banner_9월-넷째주-주일-낮예배-대표-기도문.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c

메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "footnotes": ""
}
게시물 72/80 처리 중: 주일 낮예배 대표 기도문 2월
배너 이미지 업로드 성공: banner_주일-낮예배-대표-기도문-2월.webp
미디어 업로드 리포트 로드 성공: 150개 항목
미디어 업로드 리포트 로드 성공: 150개 항목
메타 설정 실패 (접근법 1): _rank_math_description - 404
메타 설정 실패 (접근법 1): _rank_math_focus_keyword - 404
Rank Math API 메타 설정 성공 (접근법 2)
직접 포스트 업데이트를 통한 메타 설정 성공 (접근법 3)
JSON 형식 포커스 키워드 설정 실패 (접근법 4): 404 - {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
현재 포스트 메타데이터: {
  "fo